In [6]:
import json
import os
import numpy as np
import pandas as pd

test = pd.read_json('./data/val.json', typ='frame', encoding='utf-8')

tag_song = pd.read_json('./data/tag_song_pair.json', typ='frame', orient= 'table', encoding='utf-8')


song_meta = pd.read_json('./data/song_meta.json', typ='frame', encoding='utf-8')


def give_tag_song(idx):
    id_ = test.loc[idx]['id'].astype(str)
    tags = test.loc[idx]['tags']
    songs = test.loc[idx]['songs']
    return id_, tags, songs

def take_date(idx):
    target_date = test.loc[idx]['updt_date']
    year = target_date[:4]
    month = target_date[5:7]
    day = target_date[8:10]
    date = year+month+day
    return int(date)

def valid_date(idx, id_):
    plylst_date = take_date(idx)
    song_date = song_meta[song_meta['id']==id_]['issue_date'].to_list()[0]
    if song_date < plylst_date: return True
    else: return False

def tagIsTen(list): return len(list) == 50
def songIsHund(list): return len(list) == 100

def resultCase_tag_song(idx):
    id_, tags, songs = give_tag_song(idx)
    
    tag_count = [0 for _ in range(29160)]
    # 태그 빈도수
    
    for i in range(29160):
        for s in tag_song.loc[i]['song']:
            if s in songs:
                tag_count[i] += 1   

    song_count = {}
    # 곡 빈도수
    
    for s in songs:
        song_count[s] = 0
    for t in tags:
        for s in tag_song[tag_song['tag']==t]['song'].to_list()[0]:
            if s in songs:
                song_count[s] = song_count[s] + 1    
    print(song_count)
    # tag selection
    new_tags = []
    tag_count_copy = tag_count
    left_num = 50
    print(tag_count_copy)
    for i in range(left_num):
        target = tag_count_copy.index(max(tag_count_copy))
        if tag_count_copy[target] == 0: break
        if tag_song.loc[target]["tag"] not in new_tags: new_tags.append(tag_song.loc[target]['tag'])
        tag_count_copy[target] = 0   
    if tagIsTen(new_tags) is False: print("need more tag") # 유사성/제목으로 태그 추가
    del tag_count_copy 
    if tagIsTen(new_tags) is False: print("error")
    
    # song selection
    new_songs = []
    new_song_dic = {}
    left_num = 100
    for t in new_tags:
        for s in tag_song[tag_song['tag']==t]['song'].to_list()[0]:
            if s not in songs:
                if s not in new_song_dic:
                    new_song_dic[s] = 0
                else: new_song_dic[s] = new_song_dic[s]+1
    for i in range(left_num):
        if len(new_song_dic.keys()) <= 0: break
        target = max(new_song_dic.keys(), key=(lambda k: new_song_dic[k]))
        if valid_date(idx, target) and target not in new_songs: new_songs.append(target)
        del new_song_dic[target]
    if songIsHund(new_songs) is False: # 유사도로 태그 뽑기
                # 임시 코드
        new_song_dic = {}
        for i in range(29160):
            for s in tag_song.loc[i]['song']:
                if s not in songs:
                    if s not in new_song_dic:
                        new_song_dic[s] = 0
                    else: new_song_dic[s] = new_song_dic[s]+1
        left_num = 100 - len(new_songs)
        for i in range(left_num):
            if len(new_song_dic.keys()) <= 0: break
            target = max(new_song_dic.keys(), key=(lambda k: new_song_dic[k]))
            if valid_date(idx, target) and target not in new_songs: new_songs.append(target)
            del new_song_dic[target]
        if songIsHund(new_songs) is False: print("need more songs")
    print("number of tags: " + str(len(new_tags)) + ", number of songs: " + str(len(new_songs)))
    print(song_count)
    result_list = []
    result_list.append(id_)
    result_list.append(new_tags[:10])
    result_list.append(new_songs)
    return result_list

In [24]:

def resultCase_tag_song(idx):
    id_, tags, songs = give_tag_song(idx)
    
    tag_count = [0 for _ in range(29160)]
    # 태그 빈도수
    
    for i in range(29160):
        for s in tag_song.loc[i]['song']:
            if s in songs:
                tag_count[i] += 1   

    song_count = {}
    # 곡 빈도수
    
    for s in songs:
        song_count[s] = 0
    for t in tags:
        for s in tag_song[tag_song['tag']==t]['song'].to_list()[0]:
            if s in songs:
                song_count[s] = song_count[s] + 1    
    print(song_count)
    # tag selection
    new_tags = []
    tag_count_copy = tag_count
    left_num = 20
    for i in range(left_num):
        target = tag_count_copy.index(max(tag_count_copy))
        if tag_count_copy[target] == 0: break
        if tag_song.loc[target]["tag"] not in new_tags: new_tags.append(tag_song.loc[target]['tag'])
        tag_count_copy[target] = 0   
    if tagIsTen(new_tags) is False: print("need more tag") # 유사성/제목으로 태그 추가
    del tag_count_copy 
    if tagIsTen(new_tags) is False: print("error")
    
    # song selection
    new_songs = []
    new_song_dic = {}
    left_num = 100
    print("new tagg : {}".format(new_tags))
    for t in new_tags:
        for s in tag_song[tag_song['tag']==t]['song'].to_list()[0]:
            if s not in songs:
                if s not in new_song_dic:
                    new_song_dic[s] = 0
                else: new_song_dic[s] = new_song_dic[s]+1

    qwe =0
    while(qwe<=99):
        if len(new_song_dic.keys()) <= 0: break
        target = max(new_song_dic.keys(), key=(lambda k: new_song_dic[k]))
        if valid_date(idx, target) and target not in new_songs: new_songs.append(target); qwe+=1
        del new_song_dic[target]
    if songIsHund(new_songs) is False: # 유사도로 태그 뽑기
                # 임시 코드
        new_song_dic = {}
        for i in range(29160):
            for s in tag_song.loc[i]['song']:
                if s not in songs:
                    if s not in new_song_dic:
                        new_song_dic[s] = 0
                    else: new_song_dic[s] = new_song_dic[s]+1
        left_num = 100 - len(new_songs)
        for i in range(left_num):
            if len(new_song_dic.keys()) <= 0: break
            target = max(new_song_dic.keys(), key=(lambda k: new_song_dic[k]))
            if valid_date(idx, target) and target not in new_songs: new_songs.append(target)
            del new_song_dic[target]
        if songIsHund(new_songs) is False: print("need more songs")
    print("number of tags: " + str(len(new_tags)) + ", number of songs: " + str(len(new_songs)))
    print(song_count)
    result_list = []
    result_list.append(id_)
    result_list.append(new_tags[:10])
    result_list.append(new_songs)
    return result_list

In [25]:
import time
start = time.time()

asd = resultCase_tag_song(20000)

print(time.time() - start)

{69335: 2, 670105: 0, 392177: 1, 705564: 1, 339899: 1, 10190: 1, 168744: 2, 370394: 0}
need more tag
error
new tagg : ['JPOP', '일본', '일본노래', '일본음악', 'JPOP모음', '기분전환', '독특한', '명곡', '분위기', '감성', '드라이브', '비오는날', '신나는', '애니메이션OST', '인디', '잔잔한', '제이팝', '추천곡', '트렌디', '휴식']
number of tags: 20, number of songs: 100
{69335: 2, 670105: 0, 392177: 1, 705564: 1, 339899: 1, 10190: 1, 168744: 2, 370394: 0}
8.627248525619507


In [27]:
asd

['42728',
 ['JPOP', '일본', '일본노래', '일본음악', 'JPOP모음', '기분전환', '독특한', '명곡', '분위기', '감성'],
 [606596,
  288102,
  653791,
  688062,
  700090,
  580527,
  327385,
  264103,
  204900,
  209127,
  191226,
  1133,
  534974,
  45747,
  572970,
  246882,
  182116,
  379541,
  318530,
  681541,
  422736,
  165381,
  624957,
  625875,
  70506,
  365762,
  595181,
  695025,
  206932,
  239980,
  273100,
  10961,
  569405,
  471691,
  408698,
  82815,
  411372,
  608666,
  182756,
  151518,
  119974,
  548304,
  679436,
  224364,
  454218,
  126691,
  665518,
  64052,
  32906,
  482,
  459256,
  131757,
  99287,
  657261,
  624852,
  297411,
  494037,
  461341,
  592526,
  560378,
  495557,
  397712,
  561788,
  37969,
  333595,
  40175,
  336466,
  206346,
  567139,
  534695,
  502078,
  305501,
  534989,
  666228,
  174749,
  601037,
  437232,
  76888,
  11657,
  372528,
  143290,
  208962,
  274504,
  667843,
  700850,
  79590,
  112983,
  244700,
  81126,
  146989,
  278184,
  343974,
  540926,
 

In [30]:
for i in asd[2]:
    print(song_meta.loc[i]["song_name"])

スパ-クル (movie ver.) / Sparkle (movie ver.)
RAIN
RPG
前前前世 (movie ver.) / Zenzenzense (movie ver.)
なんでもないや (movie ver.) / Nandemonaiya (movie ver.)
Maboroshino Inochi (환상의 생명)
Wherever You Are
Best Friend
Some Like It Hot!! Samurai Heart (애니메이션 `은혼` 엔딩 테마곡)
Orion (NHK TV 애니메이션『3월의 라이온』2쿨 엔딩곡)
ピ&#12540;スサイン / Peace Sign
あの夏へ / Ano Natsue (어느 여름날)
Sakura
ミスト / Mist
好き / Suki (좋아)
淚色 / Namidairo (눈물의 색)
Sakuramitsutsuki (벚꽃만월) (애니메이션 `은혼` 오프닝 테마 곡)
花鳥風月 / Kachou Huugetsu (화조풍월)
TT
I`m A Believer (애니메이션 `하이큐!! 세컨드 시즌` 오프닝 테마곡)
Kataomoi
君がくれた夏 / Kimiga Kureta Natsu (네가 준 여름)
Kawaranai Mono (변하지 않는 것) (`시간을 달리는 소녀` 삽입곡)
I`m Not The Only One
Yukino Hana (박효신 `눈의 꽃` 원곡)
會いたくて 會いたくて / Aitakute Aitakute (보고 싶어서 보고 싶어서)
Last Carnival
Close To You
Hikarunara
My Friend
Marvin Gaye (Feat. Meghan Trainor)
소나기
君はロックを聽かない / Kimiwa Rock Wo Kikanai (너는 록을 듣지 않아)
오랜 날 오랜 밤
Desperado
Dragon Night
Brave Shine (페이트 / 스테이 나이트 `언리미티드 블레이드 워크스` 오프닝 테마 곡)
Good-Bye Days
4 Walls
Imagination (애니메이션 `하이큐!!` 오프닝 테마곡)
Ha